In [1]:
import sys
import os

# Get the root directory of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to the system path
sys.path.insert(0, project_root)

# Now you can import watermarking.utils
import watermarking.utils as utils

In [2]:
from pprint import pprint
import torch
from watermarking.llm import LLM, UnigramWatermarkedLLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

prompts = [
    "Základní zásady správné výživy jsou",
    "Pokud chce být člověk zdravyý, pak by měl"
    "Když se člověk cítí unavený, měl by",
    "V Praze se nachází",
    "V roce 1989 byla v Československu provedena",
    "První defenestrace v Praze se odehrála v roce",
]

# Free memory if this cell was run previously
if 'llm_model_w' in locals():
    del llm_model_w

model_name="BUT-FIT/csmpt7b"
max_lenght=128

llm_model = LLM(model_name=model_name, device=device, top_p=0.9)
texts = llm_model.generate_text(prompts, max_length=max_lenght)

del llm_model


llm_model_w = UnigramWatermarkedLLM(
    model_name=model_name, device=device, wm_strength=3, top_p=0.9, green_list_size=0.5
)
texts_w = llm_model_w.generate_text(prompts, max_length=max_lenght)


/mnt/data/xjarol06_firllm/conda/envs/llm/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/mnt/data/xjarol06_firllm/.hfcache/modules/transformers_modules/BUT-FIT/csmpt7b/538c5ba2d5cf00d02f8504e444f3552c75e1a96f/configuration_mpt.py:116: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/124 [00:00<?, ?it/s]

/mnt/data/xjarol06_firllm/conda/envs/llm/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/mnt/data/xjarol06_firllm/.hfcache/modules/transformers_modules/BUT-FIT/csmpt7b/538c5ba2d5cf00d02f8504e444f3552c75e1a96f/configuration_mpt.py:116: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/124 [00:00<?, ?it/s]

In [3]:
from viz_utils import create_highlighted_passage
from IPython.display import display, HTML

def viz_texts(text_to_viz):
    tokens = llm_model_w.tokenizer(text_to_viz, add_special_tokens=False, return_tensors='pt')['input_ids'].to(device).flatten()
    token_list = []
    for t in tokens.tolist():
        token_list.append(llm_model_w.tokenizer.decode(t))
    # Use torch.isin to check which tokens are in self.green
    binary_tensor = torch.isin(tokens, llm_model_w.green).int()
    # Convert to list if needed
    binary_list = binary_tensor.tolist()
    
    highlighted_passage = create_highlighted_passage(token_list,
                                                        binary_list,
                                                        None,
                                                        'green',
                                                        'linear')
    display(HTML("\n".join(highlighted_passage)))

for text, text_wm in zip(texts, texts_w):
    viz_texts(text)
    viz_texts(text_wm)
    print()
    

    
